In [1]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet101V2
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler

In [2]:
# Specify the input image size for 
img_size = (224, 224)

# Data Augmentation (commented out for comparison)
# train_datagen = ImageDataGenerator(
#     rescale=1./255,
#     rotation_range=20,
#     zoom_range=0.2,
#     horizontal_flip=True
# )

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Set the batch size
batch_size = 16

# Specify the paths to your train, validation, and test data
train_dir = 'D:\\Splitted Curated X-Ray Dataset\\train'
val_dir = 'D:\\Splitted Curated X-Ray Dataset\\val'
test_dir = 'D:\\Splitted Curated X-Ray Dataset\\test'

# Flow training images in batches using train_datagen generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Flow validation images in batches using val_datagen generator
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',

)

# Flow test images in batches using test_datagen generator
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=1,
    class_mode='categorical',
    shuffle = False
)

Found 7364 images belonging to 4 classes.
Found 920 images belonging to 4 classes.
Found 924 images belonging to 4 classes.


In [3]:
# Load the model
base_model = ResNet101V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Create a new model on top
model = models.Sequential()

# Add the base model
model.add(base_model)

# Flatten the output layer to 1 dimension
model.add(layers.Flatten())

# Add a fully connected layer with 256 hidden units and ReLU activation
model.add(layers.Dense(256, activation='relu'))

# Add a dropout layer for regularization
model.add(layers.Dropout(0.5))

# Add the final output layer with 4 classes for classification
model.add(layers.Dense(4, activation='softmax'))

# Print the model summary
model.summary()

171317808/171317808 [==============================] - 47s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet101v2 (Functional)    (None, 7, 7, 2048)        42626560  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 256)               25690368  
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 4)                 1028      
                                                                 
Total params: 68,317,956
Trainable params: 68,220,292
Non-trainable params: 97,664
_____________________________________

In [4]:
# Define the initial learning rate and the learning rate schedule
initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)

# Define the optimizer with the learning rate schedule
opt = optimizers.Adam(learning_rate=initial_learning_rate)

# Compile the model
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

# Set up early stopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Define a custom learning rate scheduler function
def custom_lr_schedule(epoch):
    return initial_learning_rate * (0.96 ** (epoch / 10))

# Set up LearningRateScheduler callback with the custom function
lr_scheduler = LearningRateScheduler(custom_lr_schedule)

# Train the model with the specified number of epochs
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=75,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    callbacks=[early_stop, lr_scheduler]
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator, steps=len(test_generator))
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

Epoch 1/75
460/460 [==============================] - 157s 313ms/step - loss: 1.8639 - accuracy: 0.4242 - val_loss: 1.3318 - val_accuracy: 0.3553 - lr: 0.0010
Epoch 2/75
460/460 [==============================] - 134s 291ms/step - loss: 1.8793 - accuracy: 0.4540 - val_loss: 1.5723 - val_accuracy: 0.5077 - lr: 9.9593e-04
Epoch 3/75
460/460 [==============================] - 135s 293ms/step - loss: 1.1215 - accuracy: 0.5361 - val_loss: 0.9841 - val_accuracy: 0.5888 - lr: 9.9187e-04
Epoch 4/75
460/460 [==============================] - 135s 293ms/step - loss: 0.8804 - accuracy: 0.6421 - val_loss: 0.7956 - val_accuracy: 0.6623 - lr: 9.8783e-04
Epoch 5/75
460/460 [==============================] - 135s 294ms/step - loss: 0.7473 - accuracy: 0.7016 - val_loss: 0.7988 - val_accuracy: 0.7094 - lr: 9.8380e-04
Epoch 6/75
460/460 [==============================] - 135s 294ms/step - loss: 0.7433 - accuracy: 0.7241 - val_loss: 0.5785 - val_accuracy: 0.7730 - lr: 9.7980e-04
Epoch 7/75
460/460 [======

In [5]:
from sklearn.metrics import classification_report

# Get predictions on the test set
y_pred = model.predict(test_generator)
y_true = test_generator.classes

# Convert predictions to class labels
y_pred_labels = tf.argmax(y_pred, axis=1)
class_labels = list(test_generator.class_indices.keys())

# Generate and print the classification report
print("\nClassification Report:")
print(classification_report(y_true, y_pred_labels, target_names=class_labels, digits=4))

924/924 [==============================] - 21s 21ms/step

Classification Report:
                     precision    recall  f1-score   support

           COVID-19     0.9160    0.9302    0.9231       129
             Normal     0.8970    0.9052    0.9011       327
Pneumonia-Bacterial     0.7110    0.8173    0.7604       301
    Pneumonia-Viral     0.5812    0.4072    0.4789       167

           accuracy                         0.7900       924
          macro avg     0.7763    0.7650    0.7659       924
       weighted avg     0.7820    0.7900    0.7820       924

